In [1]:
# Import all the necessary modules
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas_datareader as pdr
import math
import datetime
import itertools
import yfinance as yf
import seaborn as sn
from IPython.display import display, HTML
from trend_following import (apply_jupyter_fullscreen_css, load_financial_data, get_returns_volatility, calculate_slope, trend_signal, slope_signal, 
                             create_trend_strategy, get_close_prices, calculate_donchian_channels)
from strategy_performance_metrics import (calculate_sharpe_ratio, calculate_calmar_ratio, calculate_CAGR, calculate_risk_and_performance_metrics,
                                          calculate_compounded_cumulative_returns, estimate_fee_per_trade, rolling_sharpe_ratio)
import coinbase_utils as cn
import strategy_performance_metrics as perf
import trend_following as tf
%matplotlib inline

In [51]:
import importlib
importlib.reload(cn)
importlib.reload(perf)
importlib.reload(tf)

<module 'trend_following' from '/Users/adheerchauhan/Documents/git/trend_following/trend_following.py'>

In [3]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('Display.max_rows', None)
pd.set_option('Display.max_columns',None)
apply_jupyter_fullscreen_css()

## Helper functions to help pull the data and run the analysis

### Moving Average and Donchian Channels Signals

In [55]:
from strategy_performance_metrics import calculate_risk_and_performance_metrics

import seaborn as sns

def plot_moving_avg_crossover_performance(df_performance, ticker):
    unique_step_sizes = df_performance['stepsize'].unique()

    # Plotting each heatmap in a loop
    for step in unique_step_sizes:
        subset = df_performance[df_performance['stepsize'] == step]
        pivoted_df_sharpe = subset.pivot(index='slow_mavg', columns='fast_mavg', values='sharpe_ratio')
        pivoted_df_calmar = subset.pivot(index='slow_mavg', columns='fast_mavg', values='calmar_ratio')
        pivoted_df_return = subset.pivot(index='slow_mavg', columns='fast_mavg', values='annualized_return')
        
        fig = plt.figure(figsize=(30,6))
        # plt.style.use('bmh')
        layout = (1,3)
        sharpe_ax = plt.subplot2grid(layout, (0,0))#, colspan=2)
        calmar_ax = plt.subplot2grid(layout, (0,1))#, colspan=2)
        return_ax = plt.subplot2grid(layout, (0,2))#, colspan=2)

        sns.heatmap(pivoted_df_sharpe, annot=True, fmt=".2f", cmap='RdYlGn', linewidths=.5, ax=sharpe_ax)
        _ = sharpe_ax.set_title(f'{ticker} Sharpe Ratio Heatmap\nStep Size: {step}')
        _ = sharpe_ax.set_ylabel('Slow Moving Average (Days)')
        _ = sharpe_ax.set_xlabel('Fast Moving Average (Days)')

        sns.heatmap(pivoted_df_calmar, annot=True, fmt=".2f", cmap='RdYlGn', linewidths=.5, ax=calmar_ax)
        _ = calmar_ax.set_title(f'{ticker} Calmar Ratio Heatmap\nStep Size: {step}')
        _ = calmar_ax.set_ylabel('Slow Moving Average (Days)')
        _ = calmar_ax.set_xlabel('Fast Moving Average (Days)')

        sns.heatmap(pivoted_df_return, annot=True, fmt=".2f", cmap='RdYlGn', linewidths=.5, ax=return_ax)
        _ = return_ax.set_title(f'{ticker} Annualized Return Heatmap\nStep Size: {step}')
        _ = return_ax.set_ylabel('Slow Moving Average (Days)')
        _ = return_ax.set_xlabel('Fast Moving Average (Days)')

        plt.tight_layout()
    
    return

def generate_trend_signal_with_donchian_channel(start_date, end_date, ticker, fast_mavg, slow_mavg, mavg_stepsize, moving_avg_type='exponential', price_or_returns_calc='returns',
                                                rolling_donchian_window=20, include_signal_strength=True, long_only=False, use_coinbase_data=True):
    
    # Generate Trend Signal
    if use_coinbase_data:
        df = cn.get_coinbase_ohlc_data(ticker=ticker)
        df = (df[['close']].rename(columns={'close':ticker}))
        df = df[(df.index.get_level_values('date') >= start_date) & (df.index.get_level_values('date') <= end_date)]
    else:
        df = tf.get_close_prices(start_date, end_date, ticker, print_status=False)
    df_trend = (tf.create_trend_strategy(df, ticker, mavg_start=fast_mavg, mavg_end=slow_mavg, mavg_stepsize=mavg_stepsize, slope_window=10, moving_avg_type=moving_avg_type,
                                      price_or_returns_calc=price_or_returns_calc)
                .rename(columns={f'{ticker}_trend_strategy_returns': f'{ticker}_trend_strategy_returns_{fast_mavg}_{mavg_stepsize}_{slow_mavg}',
                                 f'{ticker}_trend_strategy_trades': f'{ticker}_trend_strategy_trades_{fast_mavg}_{mavg_stepsize}_{slow_mavg}'}))
    # Generate Donchian Signal
    df_donchian = tf.calculate_donchian_channels(start_date=start_date, end_date=end_date, ticker=ticker, price_or_returns_calc=price_or_returns_calc,
                                            rolling_donchian_window=rolling_donchian_window, use_coinbase_data=use_coinbase_data)
    if price_or_returns_calc == 'price':

        # Buy signal: Price crosses above upper band
        df_donchian[f'{ticker}_{rolling_donchian_window}_donchian_signal'] = np.where(
            (df_donchian[f'close'] > df_donchian[f'{ticker}_{rolling_donchian_window}_donchian_upper_band_price']), 1,
            np.where((df_donchian[f'close'] < df_donchian[f'{ticker}_{rolling_donchian_window}_donchian_lower_band_price']), -1, 0))
    elif price_or_returns_calc == 'returns':
        df_donchian[f'{ticker}_{rolling_donchian_window}_donchian_signal'] = np.where(
            (df_donchian[f'{ticker}_pct_returns'] > df_donchian[f'{ticker}_{rolling_donchian_window}_donchian_upper_band_returns']), 1,
            np.where((df_donchian[f'{ticker}_pct_returns'] < df_donchian[f'{ticker}_{rolling_donchian_window}_donchian_lower_band_returns']), -1, 0))

    # Merging the Trend and donchian Dataframes
    donchian_cols = [f'{ticker}_{rolling_donchian_window}_donchian_upper_band_{price_or_returns_calc}', f'{ticker}_{rolling_donchian_window}_donchian_lower_band_{price_or_returns_calc}',
                     f'{ticker}_{rolling_donchian_window}_donchian_middle_band_{price_or_returns_calc}', f'{ticker}_{rolling_donchian_window}_donchian_signal']
    df_trend = pd.merge(df_trend, df_donchian[donchian_cols], left_index=True, right_index=True, how='left')
    
    if include_signal_strength:
        # Calculate the strength of moving average crossover and donchian signal
        df_trend[f'{ticker}_donchian_band_width_{price_or_returns_calc}'] = (df_trend[f'{ticker}_{rolling_donchian_window}_donchian_upper_band_{price_or_returns_calc}'] -
                                                                             df_trend[f'{ticker}_{rolling_donchian_window}_donchian_lower_band_{price_or_returns_calc}'])
        donchian_strength = (np.abs(df_trend[f'{ticker}'] - df_trend[f'{ticker}_{rolling_donchian_window}_donchian_middle_band_{price_or_returns_calc}']) /
                             df_trend[f'{ticker}_donchian_band_width_{price_or_returns_calc}'])
        crossover_strength = np.abs(df_trend[f'{ticker}_{fast_mavg}_mavg'] - df_trend[f'{ticker}_{slow_mavg}_mavg']) / df_trend[f'{ticker}']

        df_trend[f'{ticker}_crossover_donchian_signal_strength'] = (donchian_strength + crossover_strength) / 2
        strength_threshold = 0.5
    
        # Moving Average and Donchian Channel Signal
        buy_signal = ((df_trend[f'{ticker}_{rolling_donchian_window}_donchian_signal'] == 1) &
                      (df_trend[f'{ticker}_trend_signal'] == 1) &
                      (df_trend[f'{ticker}_crossover_donchian_signal_strength'] > strength_threshold))
        sell_signal = ((df_trend[f'{ticker}_{rolling_donchian_window}_donchian_signal'] == -1) &
                       (df_trend[f'{ticker}_trend_signal'] == -1) &
                       ((df_trend[f'{ticker}_crossover_donchian_signal_strength'] > strength_threshold)))
    else:
        # Moving Average and Donchian Channel Signal
        buy_signal = ((df_trend[f'{ticker}_{rolling_donchian_window}_donchian_signal'] == 1) &
                      (df_trend[f'{ticker}_trend_signal'] == 1))
        sell_signal = ((df_trend[f'{ticker}_{rolling_donchian_window}_donchian_signal'] == -1) &
                       (df_trend[f'{ticker}_trend_signal'] == -1))
    
    if long_only:
        df_trend[f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_signal'] = (
            np.where(buy_signal, 1, 0))
    else:
        df_trend[f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_signal'] = (
            np.where(buy_signal, 1, np.where(sell_signal, -1, 0)))
        
    df_trend[f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns'] = (
        df_trend[(f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_signal')] * 
        df_trend[f'{ticker}_pct_returns'])
    df_trend[f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_trades'] = (
        df_trend[(f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_signal')].diff())
    
    return df_trend

def moving_avg_crossover_with_donchian_strategy_performance(start_date, end_date, ticker, moving_avg_type='exponential', price_or_returns_calc='returns',
                                                           rolling_donchian_window=20, include_signal_strength=True, long_only=False, use_coinbase_data=True):
    
    perf_cols = ['ticker', 'fast_mavg', 'slow_mavg', 'stepsize', 'annualized_return', 'sharpe_ratio', 'calmar_ratio', 'annualized_std_dev', 'max_drawdown', 'max_drawdown_duration',
                'hit_rate', 't_statistic', 'p_value', 'trade_count']
    df_performance = pd.DataFrame(columns=perf_cols)
    
    fast_mavg_list = np.arange(10, 101, 10)
    slow_mavg_list = np.arange(50, 501, 50)
    mavg_stepsize_list = [2, 4, 6, 8]
    performance_rows = []
    for slow_mavg in slow_mavg_list:
        for fast_mavg in fast_mavg_list:
            for mavg_stepsize in mavg_stepsize_list:
                if fast_mavg < slow_mavg:
                    df_trend = generate_trend_signal_with_donchian_channel(start_date, end_date, ticker, fast_mavg, slow_mavg, mavg_stepsize, moving_avg_type, price_or_returns_calc,
                                                                         rolling_donchian_window, include_signal_strength, long_only, use_coinbase_data)
                    performance_metrics = calculate_risk_and_performance_metrics(
                        df_trend, strategy_daily_return_col=f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns',
                        strategy_trade_count_col=f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_trades',
                        annual_trading_days=365, transaction_cost_est=0.005)
                    performance_rows.append({
                        'ticker': ticker,
                        'fast_mavg': fast_mavg,
                        'slow_mavg': slow_mavg,
                        'stepsize': mavg_stepsize,
                        'annualized_return': performance_metrics['annualized_return'],
                        'sharpe_ratio': performance_metrics['annualized_sharpe_ratio'],
                        'calmar_ratio': performance_metrics['calmar_ratio'],
                        'annualized_std_dev': performance_metrics['annualized_std_dev'],
                        'max_drawdown': performance_metrics['max_drawdown'],
                        'max_drawdown_duration': performance_metrics['max_drawdown_duration'],
                        'hit_rate': performance_metrics['hit_rate'],
                        't_statistic': performance_metrics['t_statistic'],
                        'p_value': performance_metrics['p_value'],
                        'trade_count': performance_metrics['trade_count']
                    })
                    # df_performance = df_performance.append(row, ignore_index=True)
    # Convert the list of rows to a DataFrame
    df_performance = pd.DataFrame(performance_rows, columns=perf_cols)
    
    plot_moving_avg_crossover_performance(df_performance, ticker)
    
    return df_performance

In [87]:
df = cn.get_coinbase_ohlc_data(ticker='BTC-USD')
# df = df[(df.index.get_level_values('date') >= start_date) & (df.index.get_level_values('date') <= end_date)]

In [89]:
df.head()

,low,high,open,close,volume
date,,,,,
2016-01-01,427.92,437.15,430.35,435.66,3863.277451
2016-01-02,432.41,437.56,435.67,435.40,3276.709621
2016-01-03,425.02,435.75,435.40,431.91,3904.335318
2016-01-04,431.37,435.79,431.90,433.85,5894.445723
2016-01-05,430.00,435.64,433.84,433.34,5150.109476


In [91]:
def calculate_average_true_range(start_date, end_date, ticker, price_or_returns_calc='price', rolling_atr_window=20,
                                 use_coinbase_data=True):
    if use_coinbase_data:
        df = cn.get_coinbase_ohlc_data(ticker=ticker)
        df = df[(df.index.get_level_values('date') >= start_date) & (df.index.get_level_values('date') <= end_date)]
        df.columns = [f'{ticker}_{x}' for x in df.columns]
    else:
        df = load_financial_data(start_date, end_date, ticker, print_status=False)  # .shift(1)
        df.columns = [f'{ticker}_open', f'{ticker}_high', f'{ticker}_low', f'{ticker}_close', f'{ticker}_adjclose', f'{ticker}_volume']

    if price_or_returns_calc == 'price':
        # Calculate the Exponential Moving Average (EMA)
        # df[f'{ticker}_{rolling_atr_window}_ema_price'] = df['close'].ewm(span=rolling_atr_window,
        #                                                                  adjust=False).mean()

        # Calculate the True Range (TR) and Average True Range (ATR)
        df[f'{ticker}_high-low'] = df[f'{ticker}_high'] - df[f'{ticker}_low']
        df[f'{ticker}_high-close'] = np.abs(df[f'{ticker}_high'] - df[f'{ticker}_close'].shift(1))
        df[f'{ticker}_low-close'] = np.abs(df[f'{ticker}_low'] - df[f'{ticker}_close'].shift(1))
        df[f'{ticker}_true_range_price'] = df[
            [f'{ticker}_high-low', f'{ticker}_high-close', f'{ticker}_low-close']].max(axis=1)
        df[f'{ticker}_{rolling_atr_window}_avg_true_range_price'] = df[f'{ticker}_true_range_price'].ewm(
            span=rolling_atr_window, adjust=False).mean()

    elif price_or_returns_calc == 'returns':
        # Calculate Percent Returns
        df[f'{ticker}_pct_returns'] = df[f'close'].pct_change()

        # Calculate Middle Line as the EMA of returns
        df[f'{ticker}_{rolling_atr_window}_ema_returns'] = df[f'{ticker}_pct_returns'].ewm(span=rolling_atr_window,
                                                                                           adjust=False).mean()

        # Calculate True Range based on absolute returns
        df[f'{ticker}_true_range_returns'] = df[f'{ticker}_{rolling_atr_window}_ema_returns'].abs()

        # Calculate ATR using the EMA of the True Range
        df[f'{ticker}_{rolling_atr_window}_avg_true_range_returns'] = df[f'{ticker}_true_range_returns'].ewm(
            span=rolling_atr_window, adjust=False).mean()

    return df

In [199]:
df_atr = calculate_average_true_range(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker='BTC-USD', price_or_returns_calc='price', rolling_atr_window=20,
                                     use_coinbase_data=True)

In [203]:
df_atr.head()#[[f'{ticker}_{rolling_atr_window}_avg_true_range_price']].head()

,BTC-USD_low,BTC-USD_high,BTC-USD_open,BTC-USD_close,BTC-USD_volume,BTC-USD_high-low,BTC-USD_high-close,BTC-USD_low-close,BTC-USD_true_range_price,BTC-USD_20_avg_true_range_price
date,,,,,,,,,,
2016-01-01,427.92,437.15,430.35,435.66,3863.277451,9.23,NaN,NaN,9.23,9.230000
2016-01-02,432.41,437.56,435.67,435.40,3276.709621,5.15,1.90,3.25,5.15,8.841429
2016-01-03,425.02,435.75,435.40,431.91,3904.335318,10.73,0.35,10.38,10.73,9.021293
2016-01-04,431.37,435.79,431.90,433.85,5894.445723,4.42,3.88,0.54,4.42,8.583074
2016-01-05,430.00,435.64,433.84,433.34,5150.109476,5.64,1.79,3.85,5.64,8.302781


In [93]:
## In Sample Performance
# in_sample_start_date = pd.Timestamp(2014, 9, 17).date()
in_sample_start_date = pd.Timestamp(2016, 1, 1).date()
in_sample_end_date = pd.Timestamp(2022, 12, 31).date()
out_of_sample_start_date = pd.Timestamp(2023, 1, 1).date()
out_of_sample_end_date = pd.Timestamp(2024, 9, 30).date()
full_sample_start_date = pd.Timestamp(2014, 9, 17).date()
full_sample_end_date = pd.Timestamp(2024, 9, 30).date()

In [107]:
df_atr = calculate_average_true_range(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker='BTC-USD', price_or_returns_calc='price',
                                     rolling_atr_window=20)

In [223]:
df_trend = generate_trend_signal_with_donchian_channel(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker='BTC-USD', fast_mavg=50, slow_mavg=250, 
                                                       mavg_stepsize=6, rolling_donchian_window=20, price_or_returns_calc='price', long_only=False, include_signal_strength=False)

In [225]:
df_atr = calculate_average_true_range(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker='BTC-USD', rolling_atr_window=20,
                                      use_coinbase_data=True, price_or_returns_calc='price')
# df_atr = df_atr[[f'{ticker}_{rolling_atr_window}_avg_true_range_price']]

In [227]:
df_atr.head()

,BTC-USD_low,BTC-USD_high,BTC-USD_open,BTC-USD_close,BTC-USD_volume,BTC-USD_high-low,BTC-USD_high-close,BTC-USD_low-close,BTC-USD_true_range_price,BTC-USD_20_avg_true_range_price
date,,,,,,,,,,
2016-01-01,427.92,437.15,430.35,435.66,3863.277451,9.23,NaN,NaN,9.23,9.230000
2016-01-02,432.41,437.56,435.67,435.40,3276.709621,5.15,1.90,3.25,5.15,8.841429
2016-01-03,425.02,435.75,435.40,431.91,3904.335318,10.73,0.35,10.38,10.73,9.021293
2016-01-04,431.37,435.79,431.90,433.85,5894.445723,4.42,3.88,0.54,4.42,8.583074
2016-01-05,430.00,435.64,433.84,433.34,5150.109476,5.64,1.79,3.85,5.64,8.302781


In [251]:
trend_cols = ['BTC-USD','BTC-USD_50_6_250_mavg_crossover_20_donchian_signal','BTC-USD_50_6_250_mavg_crossover_20_donchian_strategy_returns',
              'BTC-USD_50_6_250_mavg_crossover_20_donchian_strategy_trades']
df_returns = df_trend[trend_cols]
# df_returns = pd.merge(df_returns, df_atr, left_index=True, right_index=True, how='left')

In [253]:
def position_sizing(start_date, end_date, ticker, rolling_atr_window, atr_multiplier=0.01, risk_per_trade=0.02, initial_capital=15000, high_volatility_threshold=1.5,
                    use_coinbase_data=True, price_or_returns_calc='price'):
    df = calculate_average_true_range(start_date, end_date, ticker, price_or_returns_calc, rolling_atr_window, use_coinbase_data)
    df = df[[f'{ticker}_{rolling_atr_window}_avg_true_range_price']]
    df[f'{ticker}_volatility_regime'] = np.where((df[f'{ticker}_{rolling_atr_window}_avg_true_range_price'] > high_volatility_threshold *
                                                  df[f'{ticker}_{rolling_atr_window}_avg_true_range_price'].rolling(window=50).mean()), 'High', 'Low')
    df[f'{ticker}_position_size'] = (initial_capital * risk_per_trade) / (df[f'{ticker}_{rolling_atr_window}_avg_true_range_price'] * atr_multiplier)
    df[f'{ticker}_position_size'] = np.where(df[f'{ticker}_volatility_regime'] == 'High', df[f'{ticker}_position_size'] * 0.5, df[f'{ticker}_position_size'])
    df[f'{ticker}_position_size'] = df[f'{ticker}_position_size'].fillna(0)
    
    return df

In [279]:
def apply_strategy(start_date, end_date, ticker, fast_mavg, slow_mavg, mavg_stepsize, rolling_donchian_window, price_or_returns_calc='price',
                   long_only=False, include_signal_strength=False, rolling_atr_window=20, atr_multiplier=0.01, risk_per_trade=0.02, initial_capital=15000,
                   use_coinbase_data=True, high_volatility_threshold=1.5, stop_loss_multiplier=1.5, take_profit_multiplier=3):

    close_price_col = f'{ticker}'
    signal_col = f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_signal'
    returns_col = f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns'
    trades_col = f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_trades'
    position_col = f'{ticker}_position_size'
    atr_col = position_col = f'{ticker}_{rolling_atr_window}_avg_true_range_price'
    
    df = generate_trend_signal_with_donchian_channel(start_date=start_date, end_date=end_date, ticker=ticker, fast_mavg=fast_mavg, slow_mavg=slow_mavg, 
                                                             mavg_stepsize=mavg_stepsize, rolling_donchian_window=rolling_donchian_window, price_or_returns_calc=price_or_returns_calc,
                                                             long_only=long_only, include_signal_strength=include_signal_strength)
    trend_cols = [close_price_col, signal_col, returns_col, trades_col]
    df = df[trend_cols]
    
    df_position = position_sizing(start_date=start_date, end_date=end_date, ticker=ticker, rolling_atr_window=rolling_atr_window)
    df = pd.merge(df, df_position, left_index=True, right_index=True, how='left')
    
    initial_capital = 15000
    position = 0
    entry_price = None
    
    for i in range(1, len(df)):
        ## Taking a new long position
        if df[signal_col][i] == 1 and position == 0:
            position = df[position_col][i]
            entry_price = df[close_price_col][i]
            stop_loss = entry_price - (df[atr_col][i] * stop_loss_multiplier)
            take_profit = entry_price + (df[atr_col][i] * take_profit_multiplier)
        
        elif df[signal_col][i] == -1 and position > 0:
            initial_capital += position * (df[close_price_col][i] - entry_price)
            position = 0
            entry_price = None
        
        # Stop-loss and take-profit logic
        if position > 0:
            if df[close_price_col][i] <= stop_loss or df[close_price_col][i] >= take_profit:
                initial_capital += position * (df[close_price_col][i] - entry_price)
                position = 0
                entry_price = None
        
        df.at[df.index[i], 'Capital'] = initial_capital

    return df

In [281]:
df_strategy = apply_strategy(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker='BTC-USD', fast_mavg=50, slow_mavg=250, mavg_stepsize=6, rolling_donchian_window=20)

In [287]:
df_strategy['BTC-USD_strategy_position'] = df_strategy['BTC-USD_position_size'] * df_strategy['BTC-USD_50_6_250_mavg_crossover_20_donchian_signal']

In [291]:
strategy_cols = ['BTC-USD', 'BTC-USD_50_6_250_mavg_crossover_20_donchian_signal', 'BTC-USD_50_6_250_mavg_crossover_20_donchian_strategy_returns',
                'BTC-USD_50_6_250_mavg_crossover_20_donchian_strategy_trades', 'Capital', 'BTC-USD_strategy_position']

In [314]:
trend_cols = ['BTC-USD','BTC-USD_pct_returns','BTC-USD_50_mavg','BTC-USD_90_mavg','BTC-USD_130_mavg','BTC-USD_170_mavg','BTC-USD_210_mavg','BTC-USD_250_mavg',
              'BTC-USD_trend_signal','BTC-USD_20_donchian_signal','BTC-USD_50_6_250_mavg_crossover_20_donchian_signal','BTC-USD_50_6_250_mavg_crossover_20_donchian_strategy_returns',
              'BTC-USD_50_6_250_mavg_crossover_20_donchian_strategy_trades']

In [304]:
sorted(df_trend.columns)

['BTC-USD',
 'BTC-USD_130_mavg',
 'BTC-USD_130_mavg_slope',
 'BTC-USD_170_mavg',
 'BTC-USD_170_mavg_slope',
 'BTC-USD_20_donchian_lower_band_price',
 'BTC-USD_20_donchian_middle_band_price',
 'BTC-USD_20_donchian_signal',
 'BTC-USD_20_donchian_upper_band_price',
 'BTC-USD_210_mavg',
 'BTC-USD_210_mavg_slope',
 'BTC-USD_250_mavg',
 'BTC-USD_250_mavg_slope',
 'BTC-USD_50_6_250_mavg_crossover_20_donchian_signal',
 'BTC-USD_50_6_250_mavg_crossover_20_donchian_strategy_returns',
 'BTC-USD_50_6_250_mavg_crossover_20_donchian_strategy_trades',
 'BTC-USD_50_mavg',
 'BTC-USD_50_mavg_slope',
 'BTC-USD_90_mavg',
 'BTC-USD_90_mavg_slope',
 'BTC-USD_pct_returns',
 'BTC-USD_ribbon_thickness',
 'BTC-USD_trend_signal',
 'BTC-USD_trend_slope_signal',
 'BTC-USD_trend_slope_strategy_returns',
 'BTC-USD_trend_slope_strategy_trades',
 'BTC-USD_trend_strategy_returns_50_6_250',
 'BTC-USD_trend_strategy_trades_50_6_250']

In [316]:
df_trend[trend_cols].head(200)

,BTC-USD,BTC-USD_pct_returns,BTC-USD_50_mavg,BTC-USD_90_mavg,BTC-USD_130_mavg,BTC-USD_170_mavg,BTC-USD_210_mavg,BTC-USD_250_mavg,BTC-USD_trend_signal,BTC-USD_20_donchian_signal,BTC-USD_50_6_250_mavg_crossover_20_donchian_signal,BTC-USD_50_6_250_mavg_crossover_20_donchian_strategy_returns,BTC-USD_50_6_250_mavg_crossover_20_donchian_strategy_trades
date,,,,,,,,,,,,,
2016-01-11,449.19,-0.001778,443.092417,442.692652,442.538354,442.456609,442.405996,442.371577,1.0,0,0,-0.000000,NaN
2016-01-12,434.01,-0.033794,442.158208,441.877429,441.765993,441.706292,441.669101,441.643712,1.0,0,0,-0.000000,0.0
2016-01-13,432.77,-0.002857,441.250326,441.079690,441.008330,440.969322,440.944759,440.927876,1.0,0,0,-0.000000,0.0
2016-01-14,430.03,-0.006331,440.224257,440.171421,440.143355,440.126793,440.115956,440.108334,1.0,0,0,-0.000000,0.0
2016-01-15,357.53,-0.168593,433.037466,433.764251,434.023078,434.155383,434.235642,434.289498,1.0,0,0,-0.000000,0.0
2016-01-16,388.70,0.087181,429.359592,430.454304,430.851907,431.056912,431.181886,431.266016,-1.0,0,0,0.000000,0.0
2016-01-17,378.46,-0.026344,425.314304,426.822292,427.376264,427.663274,427.838714,427.957022,-1.0,0,0,-0.000000,0.0
2016-01-18,384.89,0.016990,422.225888,424.026981,424.694707,425.041947,425.254639,425.398257,-1.0,0,0,0.000000,0.0
2016-01-19,375.27,-0.024994,418.767073,420.915744,421.717672,422.135822,422.392322,422.565681,-1.0,0,0,-0.000000,0.0


In [295]:
df_strategy[strategy_cols].head(200)

,BTC-USD,BTC-USD_50_6_250_mavg_crossover_20_donchian_signal,BTC-USD_50_6_250_mavg_crossover_20_donchian_strategy_returns,BTC-USD_50_6_250_mavg_crossover_20_donchian_strategy_trades,Capital,BTC-USD_strategy_position
date,,,,,,
2016-01-11,449.19,0,-0.000000,NaN,NaN,0.000000
2016-01-12,434.01,0,-0.000000,0.0,15000.000000,0.000000
2016-01-13,432.77,0,-0.000000,0.0,15000.000000,0.000000
2016-01-14,430.03,0,-0.000000,0.0,15000.000000,0.000000
2016-01-15,357.53,0,-0.000000,0.0,15000.000000,0.000000
2016-01-16,388.70,0,0.000000,0.0,15000.000000,0.000000
2016-01-17,378.46,0,-0.000000,0.0,15000.000000,0.000000
2016-01-18,384.89,0,0.000000,0.0,15000.000000,0.000000
2016-01-19,375.27,0,-0.000000,0.0,15000.000000,0.000000


In [289]:
df_strategy.head(200)

,BTC-USD,BTC-USD_50_6_250_mavg_crossover_20_donchian_signal,BTC-USD_50_6_250_mavg_crossover_20_donchian_strategy_returns,BTC-USD_50_6_250_mavg_crossover_20_donchian_strategy_trades,BTC-USD_20_avg_true_range_price,BTC-USD_volatility_regime,BTC-USD_position_size,Capital,BTC-USD_strategy_position
date,,,,,,,,,
2016-01-11,449.19,0,-0.000000,NaN,9.955579,Low,3013.385802,NaN,0.000000
2016-01-12,434.01,0,-0.000000,0.0,10.684571,Low,2807.786949,15000.000000,0.000000
2016-01-13,432.77,0,-0.000000,0.0,10.857469,Low,2763.074812,15000.000000,0.000000
2016-01-14,430.03,0,-0.000000,0.0,10.515806,Low,2852.848485,15000.000000,0.000000
2016-01-15,357.53,0,-0.000000,0.0,16.465729,Low,1821.966113,15000.000000,0.000000
2016-01-16,388.70,0,0.000000,0.0,18.714707,Low,1603.017344,15000.000000,0.000000
2016-01-17,378.46,0,-0.000000,0.0,18.741878,Low,1600.693394,15000.000000,0.000000
2016-01-18,384.89,0,0.000000,0.0,18.480747,Low,1623.311041,15000.000000,0.000000
2016-01-19,375.27,0,-0.000000,0.0,18.196866,Low,1648.635543,15000.000000,0.000000


In [255]:
df_position = position_sizing(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker='BTC-USD', rolling_atr_window=20)

In [257]:
df_position.head()

,BTC-USD_20_avg_true_range_price,BTC-USD_volatility_regime,BTC-USD_position_size
date,,,
2016-01-01,9.230000,Low,3250.270856
2016-01-02,8.841429,Low,3393.116820
2016-01-03,9.021293,Low,3325.465829
2016-01-04,8.583074,Low,3495.251161
2016-01-05,8.302781,Low,3613.247001


In [259]:
df_returns.shape

(2547, 4)

In [261]:
df_returns = pd.merge(df_returns, df_position, left_index=True, right_index=True, how='left')

In [263]:
df_returns.shape

(2547, 7)

In [271]:
df_returns[df_returns['BTC-USD_volatility_regime'] == 'High']#.head(20)

,BTC-USD,BTC-USD_50_6_250_mavg_crossover_20_donchian_signal,BTC-USD_50_6_250_mavg_crossover_20_donchian_strategy_returns,BTC-USD_50_6_250_mavg_crossover_20_donchian_strategy_trades,BTC-USD_20_avg_true_range_price,BTC-USD_volatility_regime,BTC-USD_position_size
date,,,,,,,
2016-05-28,523.25,1,0.110298,0.0,14.872770,High,1008.554578
2016-05-29,525.22,1,0.003765,0.0,19.073458,High,786.433154
2016-05-30,532.55,1,0.013956,0.0,19.352177,High,775.106611
2016-05-31,531.34,0,-0.000000,-1.0,19.879588,High,754.542788
2016-06-01,534.84,1,0.006587,1.0,19.615818,High,764.688986
2016-06-02,537.87,1,0.005665,0.0,18.637169,High,804.843281
2016-06-03,571.25,1,0.062060,0.0,20.866010,High,718.872471
2016-06-04,576.31,1,0.008858,0.0,21.769247,High,689.045425
2016-06-05,575.17,0,-0.000000,-1.0,21.103604,High,710.779057


In [267]:
df_returns.groupby(['BTC-USD_volatility_regime']).size()

BTC-USD_volatility_regime
High     348
Low     2199
dtype: int64

In [123]:
# Simulate trading logic
position = 0
entry_price = 0
fast_mavg = 50
slow_mavg = 250
mavg_stepsize = 6
rolling_donchian_window = 20
total_capital = 15000
risk_per_trade = 0.05
stop_loss_multiplier = 2
ticker = 'BTC-USD'
rolling_atr_window = 20


for i in range(1, len(df_returns)):
    current_price = df_returns[f'{ticker}_close'].iloc[i]
    atr = df_returns[f'{ticker}_{rolling_atr_window}_avg_true_range_price'].iloc[i]
    signal = df_returns[f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_signal'].iloc[i]

    # Calculate stop-loss distance based on ATR
    stop_loss_distance = stop_loss_multiplier * atr
    risk_per_unit = current_price - (current_price - stop_loss_distance)

    # Calculate position size based on the risk
    position_size = (total_capital * risk_per_trade) / risk_per_unit

    if signal == 1 and position == 0:
        # Buy signal: go long
        position = position_size
        entry_price = current_price
        print(f"Buy: {position_size:.4f} units at {current_price:.2f} USD")

    elif signal == -1 and position > 0:
        # Sell signal: exit long position
        print(f"Sell: Closing long position at {current_price:.2f} USD")
        position = 0  # Reset position

    elif signal == -1 and position == 0:
        # Short signal: go short
        position = -position_size
        entry_price = current_price
        print(f"Short: {position_size:.4f} units at {current_price:.2f} USD")

    elif signal == 1 and position < 0:
        # Exit short position if a buy signal occurs
        print(f"Buy to cover: Closing short position at {current_price:.2f} USD")
        position = 0  # Reset position

Buy: 45.9130 units at 425.30 USD
Sell: Closing long position at 6188.00 USD
Short: 2.0285 units at 6154.69 USD
Buy to cover: Closing short position at 8860.23 USD
Buy: 0.8133 units at 8975.00 USD
Sell: Closing long position at 37644.10 USD
Short: 0.1971 units at 36537.22 USD


In [105]:
df_returns.head(20)

,BTC-USD,BTC-USD_50_6_250_mavg_crossover_20_donchian_signal,BTC-USD_50_6_250_mavg_crossover_20_donchian_strategy_returns,BTC-USD_50_6_250_mavg_crossover_20_donchian_strategy_trades,BTC-USD_low,BTC-USD_high,BTC-USD_open,BTC-USD_close,BTC-USD_volume,BTC-USD_high-low,BTC-USD_high-close,BTC-USD_low-close,BTC-USD_true_range_price,BTC-USD_20_avg_true_range_price
date,,,,,,,,,,,,,,
2016-01-11,449.19,0,-0.0,NaN,445.88,452.65,449.99,449.19,5597.637183,6.77,2.66,4.11,6.77,9.955579
2016-01-12,434.01,0,-0.0,0.0,431.83,449.44,449.26,434.01,6596.945453,17.61,0.25,17.36,17.61,10.684571
2016-01-13,432.77,0,-0.0,0.0,425.00,437.50,434.01,432.77,9009.150709,12.50,3.49,9.01,12.50,10.857469
2016-01-14,430.03,0,-0.0,0.0,428.00,435.27,432.70,430.03,5673.632962,7.27,2.50,4.77,7.27,10.515806
2016-01-15,357.53,0,-0.0,0.0,357.30,430.29,430.04,357.53,28641.673586,72.99,0.26,72.73,72.99,16.465729
2016-01-16,388.70,0,0.0,0.0,350.92,391.00,357.59,388.70,17985.238784,40.08,33.47,6.61,40.08,18.714707
2016-01-17,378.46,0,-0.0,0.0,372.00,391.00,388.70,378.46,8278.401793,19.00,2.30,16.70,19.00,18.741878
2016-01-18,384.89,0,0.0,0.0,370.10,386.10,378.47,384.89,8711.459756,16.00,7.64,8.36,16.00,18.480747
2016-01-19,375.27,0,-0.0,0.0,370.00,385.50,384.79,375.27,8133.367010,15.50,0.61,14.89,15.50,18.196866


In [97]:
df_atr.head(20)

,BTC-USD_low,BTC-USD_high,BTC-USD_open,BTC-USD_close,BTC-USD_volume,BTC-USD_high-low,BTC-USD_high-close,BTC-USD_low-close,BTC-USD_true_range_price,BTC-USD_20_avg_true_range_price
date,,,,,,,,,,
2016-01-01,427.92,437.15,430.35,435.66,3863.277451,9.23,NaN,NaN,9.23,9.230000
2016-01-02,432.41,437.56,435.67,435.40,3276.709621,5.15,1.90,3.25,5.15,8.841429
2016-01-03,425.02,435.75,435.40,431.91,3904.335318,10.73,0.35,10.38,10.73,9.021293
2016-01-04,431.37,435.79,431.90,433.85,5894.445723,4.42,3.88,0.54,4.42,8.583074
2016-01-05,430.00,435.64,433.84,433.34,5150.109476,5.64,1.79,3.85,5.64,8.302781
2016-01-06,428.15,433.46,433.32,430.87,5476.959959,5.31,0.12,5.19,5.31,8.017755
2016-01-07,430.64,460.15,430.66,459.07,13907.201729,29.51,29.28,0.23,29.51,10.064635
2016-01-08,447.53,464.40,459.07,454.44,8347.095040,16.87,5.33,11.54,16.87,10.712765
2016-01-09,447.66,456.00,454.41,450.38,4247.639651,8.34,1.56,6.78,8.34,10.486787
